In [1]:
import requests
from datetime import datetime

import os
from apiclient.http import MediaFileUpload, MediaIoBaseDownload
from google.oauth2 import service_account
from googleapiclient.discovery import build

def setup_gdrive(credentials_secret):
    with open('credentials.json', 'w') as f:
        f.write(os.environ[credentials_secret])
        
    # Load the credentials from the JSON key file
    credentials = service_account.Credentials.from_service_account_file(
        'credentials.json',
        scopes=['https://www.googleapis.com/auth/drive']
    )

    drive_service = build('drive', 'v3', credentials=credentials)
    return drive_service



In [2]:
with open('enre-414003-3724f64a117f.json') as f:
    credentials = f.read()

os.environ['GDRIVE_SECRET'] = credentials

In [3]:
drive_service = setup_gdrive('GDRIVE_SECRET')

In [4]:
import os

def download_file(file, path, skip_existing=False):
    file_id = file['id']
    if skip_existing and os.path.exists(os.path.join(path, file['name'])):
        return False
    
    request = drive_service.files().get_media(fileId=file_id)    
    with open(os.path.join(path, os.path.basename(file['name'])), 'wb') as fh:
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
            return True

In [7]:
1

1

In [8]:
from functools import partial
from tqdm.notebook import tqdm
from multiprocessing.dummy import Pool


def download_all_files(path, skip_existing):
    response = drive_service.files().list().execute()
    
    files = response['files']
    page_no = 1
    while response.get('nextPageToken'):
        downloaded = 0
        with Pool(10) as p:
            downloaded += sum(
                tqdm(p.imap_unordered(partial(download_file, path=path, skip_existing=skip_existing), files))
            )
        
        print(f'Downloaded {downloaded} files from page {page_no}')
        
        response = drive_service.files().list(pageToken=response['nextPageToken']).execute()
        files = response['files']
        page_no += 1
        
    print('Finished Downloading')
    
    
def download_all_files_seq(path, skip_existing):
    response = drive_service.files().list().execute()
    
    files = response['files']
    page_no = 1
    while response.get('nextPageToken'):
        downloaded = 0
        for file in tqdm(files):
            downloaded += download_file(file, path, skip_existing=skip_existing)
        print(f'Downloaded {downloaded} files from page {page_no}')
        
        response = drive_service.files().list(pageToken=response['nextPageToken']).execute()
        files = response['files']
        page_no += 1
        
    print('Finished Downloading')

In [11]:
import os

if not os.path.exists('data'): os.makedirs('data')
    
download_all_files_seq('data', skip_existing=True)

OSError: [Errno 65] No route to host

In [7]:
!ls data | wc -l

    2557


In [13]:
ls data

clima-enre-2024-02-11T13.txt
clima-enre-2024-02-11T13.txt.gz
clima-enre-2024-02-11T14.txt.gz
clima-enre-2024-02-11T15.txt.gz
clima-enre-2024-02-11T16.txt.gz
clima-enre-2024-02-11T17.txt.gz
clima-enre-2024-02-11T18.txt.gz
clima-enre-2024-02-11T19.txt.gz
clima-enre-2024-02-11T20.txt.gz
clima-enre-2024-02-11T22.txt.gz
clima-enre-2024-02-11T23.txt.gz
clima-enre-2024-02-12T00.txt.gz
clima-enre-2024-02-12T01.txt.gz
clima-enre-2024-02-12T02.txt.gz
clima-enre-2024-02-12T03.txt.gz
clima-enre-2024-02-12T04.txt.gz
clima-enre-2024-02-12T05.txt.gz
clima-enre-2024-02-12T06.txt.gz
clima-enre-2024-02-12T07.txt.gz
clima-enre-2024-02-12T08.txt.gz
clima-enre-2024-02-12T09.txt.gz
clima-enre-2024-02-12T10.txt.gz
clima-enre-2024-02-12T11.txt.gz
clima-enre-2024-02-12T12.txt.gz
clima-enre-2024-02-12T13.txt.gz
clima-enre-2024-02-12T14.txt.gz
clima-enre-2024-02-12T15:09.txt.gz
clima-enre-2024-02-12T15:30.txt.gz
clima-enre-2024-02-12T20:11.txt.gz
clima-enre-2024-02-12T21:50.txt.gz
clima-enre-2024-02-13T20:10.txt